# Árboles de decisión / Aprendizaje en conjunto (ensamble learning)

Los árboles de decisión (decision trees), sus derivaciones: *random forest* y el múltiple ganador de competiciones ([Kaggle](https://www.kaggle.com/)) *Gradient boosted tree* , se verán durante esta práctica. Los árboles de decisión son simples, fáciles de interpretar y pueden dar muy buenos resultados cuando se juntan (bagged) y se aumentan (boosted).

## Árboles de decisión de regresión

Pueden aplicarse tanto a la regresión como a la clasificación. Los árboles de decisión son gráficos en los que se puede comenzar desde la "raíz" y se recorre el camino hacia abajo tomando decisiones en las "ramas" antes de terminar finalmente en una "hoja" que da la predicción.

<img src="images/Clases_T9_T10_ARBOLES.png" alt="Drawing" style="width: 500px;"/>

Dos pasos generales para construir un árbol de decisión (regresión) son:

1. Dividir el espacio del predictor en un númmero $j$ de regiones.
2. Para cada región, se encuentra la respuesta media y se usa como el valor predicho. Esto minimizará el error cuadrado para esa región.

Las regiones en teoría se pueden dividir de cualquier manera que se elija, pero en la práctica, se dividen en rectángulos de gran dimensión (ver la Figura). Por ejemplo, se podría haber usado una línea con una pendiente distinta de cero para dividir la región de abajo para obtener un ajuste más preciso, pero la simplicidad gana aquí y simplementese se dividen n líneas horizontales y verticales o "rectángulos de alta dimensión".

<img src="images/Clases_T9_T10_Region.png" alt="Drawing" style="width: 500px;"/>


## ¿Cómo se obtienen las ramas?
Se podría tratar de construir todos los árboles imaginables y encontrar el árbol con el error cuadrado más bajo, pero esto es computacionalmente inviable, incluso para un número relativamente pequeño de predictores. En cambio, se usa un enfoque ambicioso al construir el árbol una rama a la vez. La primera rama se construye probando diferentes divisiones binarias de los datos.

Por ejemplo, $ X_1 <5 $ y $ X_1> = 5 $ serían una división potencial. $ X_2 = YES $ y $ X_2 = NO $ podría ser otra división binaria. El error cuadrado más bajo se considera para la mejor división y esa división se elige para la primera rama. Este proceso continúa ahora para cada rama, de manera interactiva, hasta que se cumplan algunos criterios de detención (número máximo de ramas, número mínimo de observaciones en una determinada rama, etc.).

### El algoritmo de entrenamiento CART 

Scikit-Learn utiliza el algoritmo CART (classification and regression trees) para entrenar Árboles de Decisión (también llamados árboles "en crecimiento"). La idea es bastante simple: el algoritmo primero divide el conjunto de entrenamiento en dos subconjuntos usando una única característica $k$ y un umbral $t_k$ (por ejemplo, "Years < 4,5"). ¿Cómo elige $k$ y $t_k$? Busca el par ($k$, $t_k$) que produce los subconjuntos más puros (ponderados por su tamaño). La función de costo que el algoritmo intenta minimizar viene dada por:

<img src="images/CART_eq_reg.png" alt="Drawing" style="width: 500px;"/>


Donde $m_{left}, m_{right}$ es el número de instancias en el subcojunto de la izquierda/derecha del árbol.

Una vez que el conjunto de entrenamiento se ha dividido con éxito en dos, se divide los subconjuntos usando la misma lógica, luego los subconjuntos secundarios y así sucesivamente, de forma recursiva. El algoritmo deja de actuar recursivamente, una vez que alcanza la profundidad máxima (definida por el hiperparámetro max_depth), o si no puede encontrar una división que reduzca la impureza. Otros **hiperparámetros** que controlan algunas condiciones adicionales de detención son: *min_samples_split, min_samples_leaf, min_weight_fraction_leaf, y max_leaf_nodes.*

## Árboles de decisión de clasificación

El árbol de clasificación es similar al de la regresión pero ahora se trata de predecir una *variable cualitativa*: la clase a la que pertenece un objeto. Ahora el objetivo es que cada región quede definida por la *clase que ocurra con mayor frecuencia* para las observaciones del conjunto de training en la región en cuestión.

Para construir el árbol, se usa también la *división binaria recursiva* y *el algoritmo CART*, pero no puede usarse el criterio RSS para hacer las particiones.

Una alternativa natural es la proporción de errores de clasificación, es decir, la proporción de observaciones del conjunto de training en la región obtenida en el nodo m que no pertenecen a la clase más frecuente.

<img src="images/Clases_T9_T10_ArbolesClass.png" style="float: left; width: 48%; margin-right: 0%; margin-bottom: 0.5em;">
<img src="images/Clases_T9_T10_construccion_Gini.png" style="float: right; width: 48%; margin-right: 2%; margin-bottom: 0.5em;">
<p style="clear: both;">

El proceso anterior se va repitiendo recursivamente, separando los subconjuntos usando la misma idea, hasta que se alcanza un criterio de parada o no puede encontrarse una división que reduzca el índice de impureza.

El algoritmo deja de actuar recursivamente, una vez que alcanza la profundidad máxima (definida por el hiperparámetro max_depth), o si no puede encontrar una división que reduzca la impureza. Otros **hiperparámetros** que controlan algunas condiciones adicionales de detención son: *min_samples_split, min_samples_leaf, min_weight_fraction_leaf, y max_leaf_nodes.*

### Ventajas de los árboles de decisión
* Los árboles son muy fáciles de explicar. Siguen una lógica cercana a la forma humana de razonar. 

* Son fáciles de interpretar especialmente si son pequeños. Admiten una visualización gráfica intuitiva. 

* Son modelos del tipo “caja blanca”. Puede seguirse fácilmente la influencia de los distintos descriptores en el proceso de predicción. 

* Conviven fácilmente predictores cuantitativos y cualitativos sin necesidad de introducir variables ficticias (dummy). 

### Desventajas de los árboles de decisión
* El entrenamiento (creación del árbol) es computacionalmente costoso puesto que, en cada nodo, el proceso de división requiere comparar todos los descriptores. Esto es especialmente notable cuando el número de descriptores y/o el de observaciones son grandes.

* No obstante, la realización de predicciones es rápida porque atravesar cada nodo implica un solo descriptor. 

* Los árboles necesitan muy pocas hipótesis sobre los datos y no tienen parámetros. Si no se imponen restricciones, la estructura se adapta a los datos del conjunto de entrenamiento. Esto puede implicar “overfitting”. 

### Reducción de la complejidad (regularización)

Para evitar el overfitting con el conjunto de training, suelen introducirse *restricciones* en la construcción del árbol de decisión, es decir alguna forma de *regularización* asociada a **hiperparámetros**.

Particularmente, la clase `sklearn.tree.DecisionTreeClassifier` tiene algunos hiperparámetros que restringen la forma del árbol de decisión:
* `min_samples_split` (el número mínimo de muestras que un nodo debe tener antes de poder dividirlo)
* `min_samples_leaf` (el número mínimo de muestras que debe tener un nodo hoja)
* `min_weight_fraction_leaf` (igual que min_samples_leaf pero expresada como una fracción del número total de instancias ponderadas)
* `max_leaf_nodes` (número máximo de nodos hoja)
* `max_features` (número máximo de características que se evalúan para la división en cada nodo).

Aumentar los hiperparámetros que empiezan por `min_` o reducir los `max_` regularizará el modelo.



## Ensemble learning
Los árboles de decisión no tienen el mismo nivel de exactitud en la predicción en comparación con otros métodos de clasificación y regresión. Existen técnicas para agrupar diferentes árboles de decisión (o cualquier clasificador) y así mejorar la capacidad predictiva, tales como:
 
* Bagging
    * Random forest
* (Gradient) Boosting
* Otras: Voting, Adaboost, Stacking, ...

### Bagging

Un único árbol profundo produce bajo sesgo pero una alta varianza. La idea ahora es:

* Separar conjunto de entrenamiento aleatoriamente en diferentes subconjuntos
* Entrenar un árbol para cada subconjunto
* Combinar los resultados

<img src="images/Clases_T9_T10_Bagging1.png" alt="Drawing" style="width: 500px;"/>
<img src="images/Clases_T9_T10_Bagging2.png" style="float: left; width: 48%; margin-right: 0%; margin-bottom: 0.5em;">
<img src="images/Clases_T9_T10_Bagging3.png" style="float: right; width: 48%; margin-right: 2%; margin-bottom: 0.5em;">
<p style="clear: both;">

#### Validación mediante *Out-of-bag error*
Cada muestra *boostrap* contendrá en promedio 2/3 de los datos (debido al muestreo con reemplazo). Se puede usar el restante 1/3 como conjunto de validación y guardar los errores para cada punto que no esté en la muestra.

#### Variable importance
Lo maravilloso de los árboles de decisión es su facilidad de interpretación. El *bagging* reduce enormemente la interpretación ya que se está promediando decisiones. Se puede usar la cantidad promedio de la disminución en la RSS (residual sum square) o el índice de Gini, cada vez que ocurre una división con un predictor particular. En general, los predictores que se utilizan más arriba en los árboles provocarán una disminución más masiva en la RSS o el índice de Gini.

### Random forest (un caso de Bagging)

* Supongamos que un descriptor (o unos pocos) es muy fuerte en comparación  con los otros. Entonces, en el conjunto de árboles agregados, la mayoría usarán este descriptor en la división del primer nodo. De esta forma los árboles serán parecidos y sus predicciones estarán muy correlacionadas.
* Promediar variables altamente correlacionadas a una reducción de la varianza tan marcada como en el caso de variables altamente no correlacionadas. 
* Por tanto, el bagging no reducirá la varianza como sería deseable.

En Random forest se sigue utilizando el bagging con un número B de árboles. Random forest soluciona el problema anterior eligiendo, en cada división, solo un subconjunto con un número m de los p predictores elegidos al azar.
Solo uno de estos m predictores se usa en la división.  Habitualmente se escoge $m \approx	\sqrt{p}$. Lo que hace que se escoja un número pequeño de descriptores.

## Gradient Boosting

### Gradient Boosting Regression
Se puede usar Boosting (como Bagging) para muchos tipos de algoritmos de aprendizaje diferentes. En este método los árboles crecen secuencialmente. Cada árbol crece usando información de un árbol anterior. El método de Boosting aprende lentamente. Un árbol se ajusta a los *residuos* del modelo y no al resultado $Y$. Cada árbol puede ser pequeño con solo unos pocos nodos terminales. Los árboles nuevos se siguen enfocando en áreas donde el modelo tiene un rendimiento bajo. Para problemas de clasificación, las log-odds se utilizan para encontrar los residuos.

Particularmente, para el caso de regresión (para Clasificación es algo similar, pero más complejo) el algoritmo (simplificado) de Gradient Boost es:

1. Se establece $ \hat{f}(x)=0 $ y $r_i = y_i$ para todos los $i$ en el training set.
2. Para $b=1,2,...,B$, se repite:

    a. Se ajuste un árbol $\hat{f^b}$ con $d$ divisiones (o $d+1$ nodos terminales u hojas) al training set **(X,r)**.
    
    b. Se actualiza $\hat{f}$ añadiendo una versión reducida (shrunken) del nuevo árbol: $$ \hat{f}(x) \gets \hat{f}(x) + \eta  \hat{f^b}(x)  $$
    
    c. Se actualizan los residuos: $$ r_i \gets r_i - \eta \hat{f^b}(x_i)   $$
    
3. La salida del modelo potenciado (boosted) es: $$ \hat{f}(x) = \sum_{b=1}^{B}{\eta \hat{f^b}(x)} $$

Los hiperparámetros del modelo son:

1. **La cantidad de árboles B**. A diferencia de Bagging y random forest, Boosting puede tener *overfitting* si B es demasiado grande, aunque este sobreajuste tiende a ocurrir lentamente, si es que lo hace. Se usa validación cruzada para seleccionar B.

2. **El parámetro de contracción (shrinkage) $ \eta $**, también llamado *learning rate*, es un pequeño número positivo que controla la velocidad a la que aprende el modelo. Los valores típicos son 0.1, 0.01 o 0.001, y la elección correcta puede depender del problema. Un valor muy pequeño $\eta$ puede requerir el uso de un valor muy grande de B para lograr un buen rendimiento.

3. **El número *d* de divisiones en cada árbol**, que controla la complejidad del conjunto potenciado. A menudo *d = 1* funciona bien, en cuyo caso cada árbol es un muñón (stump), que consiste en una sola división. En este caso, el conjunto potenciado (boosted ensemble) esta ajustando un modelo aditivo, ya que cada término implica una sola variable. De manera más general, *d* es la *profundidad* de interacción y controla el orden de interacción del modelo potenciado, ya que las *d* divisiones pueden implicar como máximo *d* variables.

4. El modelo completo (y más complejo) se puede extender aplicando regularización con la norma L1 (parámetro $\alpha$) y/o la norma L2 (parámetro $\lambda$). También, se puede involucrar un tipo de regularización para controlar la mínima pérdida requerida para realizar una partición sobre un nodo del árbol (parámetro $\gamma$). Este último parámetro también se puede utilizar para hacer *prunning*.

### Gradient Boosting Classification

El Boosting es una técnica secuencial que funciona según el principio de ensamblaje. Combina un conjunto de máquinas (learners) débiles y ofrece una precisión de predicción mejorada. En cualquier instante t, los resultados del modelo se ponderan en función de los resultados del instante t-1 anterior. Los resultados predichos correctamente reciben un peso menor y los mal clasificados se ponderan más. Lo anteriormente descrito es para clasificación, pero en regresión la idea es muy similar.
![algo](images/boosting_classification.png)

Observaciones:

1. Recuadro 1: Salida de la primera máquina débil (de la izquierda)
   - Inicialmente todos los puntos tienen el mismo peso (denotado por su tamaño).
   - El límite de decisión predice 2 puntos **+** y 5 puntos **-** correctamente.
2. Recuadro 2: Salida de la segunda máquina débil
   - Los puntos clasificados correctamente en el recuadro 1 tienen un peso inferior y viceversa.
   - El modelo se centra ahora en los puntos de alto peso y los clasifica correctamente. Pero, otros están mal clasificados ahora.

Una tendencia similar se puede ver en el recuadro 3 también. Esto continúa por muchas iteraciones. Al final, todos los modelos tienen un peso en función de su precisión y se genera un resultado consolidado.

## Hiperparámetros de Gradient Boosting

Se pueden dividir en tres categorias:

1. **Hiperparámetros específicos de los árboles de decisión:** afectan a cada árbol de forma individual en el modelo
2. **Hiperparámetros de Boosting**: afectan a la operación de boosting en el modelo.
3. **Paramétros varios**: otros parámetros para el funcionamiento general

### Hiperparámetros específicos de los árboles de decisión

#### 1. min_samples_split
- Define el número mínimo de muestras (u observaciones) que se requieren en un nodo para ser considerado para la división.
- Se utiliza para controlar el ajuste excesivo. Los valores más altos impiden que un modelo aprenda relaciones que podrían ser muy específicas para la muestra particular seleccionada para un árbol.
- Los valores demasiado altos pueden llevar a una adaptación insuficiente, por lo tanto, debe ajustarse utilizando CV.

#### 2. min_samples_leaf
- Define las muestras mínimas (u observaciones) requeridas en un nodo terminal u hoja.
- Se utiliza para controlar el ajuste excesivo similar a min_samples_split.
- En general, se deben elegir valores más bajos para los problemas de clase desequilibrados porque las regiones en las que la mayoría de la clase minoritaria será muy pequeña.

#### 3. min_weight_fraction_leaf
- Similar a min_samples_leaf pero definido como una fracción del número total de observaciones en lugar de un entero.
- Solo uno de los #2 y #3 debe ser definido.

#### 4. max_depth
- La profundidad máxima de un árbol.
- Utilizado para controlar el ajuste excesivo, ya que una mayor profundidad permitirá al modelo aprender relaciones muy específicas para una muestra en particular.
- Debe ser tuneado utilizando CV.

#### 5. max_leaf_nodes
- El número máximo de nodos terminales u hojas en un árbol.
- Se puede definir en lugar de max_depth. Dado que se crean árboles binarios, una profundidad de 'n' produciría un máximo de 2 ^ n hojas.
- Si se define este hiperparámetro se ignorará max_depth.

#### 5. max_features
- El número de características a considerar al buscar una mejor división. Estos serán seleccionados al azar.
- Como regla general, la raíz cuadrada del número total de funciones funciona muy bien, pero debemos verificar hasta el 30-40% del número total de funciones.
- Los valores más altos pueden llevar a un ajuste excesivo, pero dependen de cada caso.

Antes de pasar a otros parámetros, veamos el pseudocódigo general del algoritmo GBM para 2 clases:

```
  1. Inicializar la variable respuesta.
 2. Itera de 1 a número total de árboles
   2.1 Actualizar las ponderaciones para los objetivos en función de la ejecución anterior (mayor para las clasificadas incorrectamente)
   2.2 Ajustar el modelo a la submuestra de datos seleccionada.
   2.3 Hacer predicciones sobre el conjunto completo de observaciones.
   2.4 Actualizar la salida con los resultados actuales teniendo en cuenta la tasa de aprendizaje
 3. Devuelve la salida final. 
```

Esta es una explicación extremadamente simplificada de como funciona el gradient boosting. Los parámetros que hemos considerado hasta ahora afectarán al paso 2.2, es decir, la construcción del modelo. Consideremos otro conjunto de parámetros para gestionar el boosting:

### Hiperparámetros de Boosting

#### 1. learning_rate 
- Determina el impacto de cada árbol en el resultado final (paso 2.4). Gradient boosting funciona comenzando con una estimación inicial que se actualiza utilizando la salida de cada árbol. El parámetro de aprendizaje controla la magnitud de este cambio en las estimaciones.
- Generalmente se prefieren valores bajos, ya que hacen que el modelo sea robusto a las características específicas del árbol y, por lo tanto, le permite generalizarse bien.
- Valores bajos de la tasa de aprendizaje requerirían un mayor número de árboles para modelar todas las relaciones y serán computacionalmente costosos.

#### 2. n_estimators
- El número de árboles secuenciales a modelar (paso 2)
- A pesar de que el gradient boosting es bastante robusto con un mayor número de árboles, aún puede sobreajustarse en un punto. Por lo tanto, debe ajustarse utilizando CV para una tasa de aprendizaje particular.

#### 3. subsample
- La fracción de observaciones a seleccionar para cada árbol. La selección se realiza mediante muestreo aleatorio.
- Los valores ligeramente inferiores a 1 hacen que el modelo sea robusto al reducir la varianza.
- Los valores típicos ~ 0,8 generalmente funcionan bien, pero pueden ajustarse aún más.

### Otros hiperparámetros

#### 1. loss 
- Se refiere a la función de pérdida que debe minimizarse en cada división.
- Puede tener varios valores para el caso de clasificación y regresión. Generalmente los valores predeterminados funcionan bien. Se deben elegir otros valores solo si comprende su impacto en el modelo.

#### 2. init 
- Afecta a la inicialización de la salida.
- Se puede utilizar si hemos creado otro modelo cuyo resultado se utilizará como las estimaciones iniciales para gradient boosting.

#### 3. random_state
- La semilla de números aleatorios para que siempre se generen los mismos números aleatorios.
- Esto es importante para el ajuste de parámetros. Si no arreglamos el número aleatorio, tendremos resultados diferentes para ejecuciones posteriores en los mismos parámetros y será difícil comparar modelos.
- Potencialmente, puede resultar en un ajuste excesivo para una muestra aleatoria particular seleccionada. Podemos intentar ejecutar modelos para diferentes muestras aleatorias, lo que es computacionalmente costoso y generalmente no se usa.

#### 4. verbose
- El tipo de salida que se imprimirá cuando el modelo se ajuste. Los diferentes valores pueden ser:
  - 0: sin salida generada (por defecto)
  - 1: salida generada para árboles en ciertos intervalos
  - > 1: salida generada para todos los árboles

#### 5. warm_start
- Este hiperparámetro tiene una aplicación interesante y puede ayudar mucho si se usa judicialmente.
- Usandolo, podemos ajustar árboles adicionales en los ajustes previos de un modelo. Puede ahorrar mucho tiempo y debería explorar esta opción para aplicaciones avanzadas

#### 5. presort
- Seleccione si desea ordenar los datos para realizar splits más rápidos.
- Realiza la selección automáticamente de forma predeterminada, pero se puede cambiar si es necesario.

## Hiperparámetros de XGBoost 

Los hiperparámetros se han dividido en 3 categorías por los autores de XGBoost:

1. Hiperparámetros generales: guia el funcionamiento general.
2. Hiperparámetros de boosting: guia el paso individual de boosting
3. Hiperparámetros de aprendizaje: guia la optimización realizada

#### 1. Hiperparámetros generales

- *booster* = [gbtree|gblinear] - which booster to use.
- *silent* = [0|1] - 0 prints running messages, 1 means silent mode
- *nthread* - number of threads to use. Maximum by default

#### 2. Hiperparámetros de boosting (trees)

- *eta* = [0 .. 0.3 .. 1] - step shrinkage used in update to prevents overfitting. After each boosting step eta shrinks instance weights. Lower value makes learning process more conservative (slower learning),
- *gamma* = [0 .. ∞] - minimum loss reduction required to make a further partition on a leaf node of the tree. Higher value makes the algorithm more conservative,
- *max_depth* = [1 .. 6 .. ∞] - maximum depth of each tree,
- *min_child_weight* = [0 .. 1 .. ∞] - minimum sum of instance weight needed in a tree node. Further partitioning from that node is abandoned when a sum is not obtained. Higher value makes the algorithm more conservative.
- *max_delta_step* = [0 .. ∞] - maximum delta step we allow each tree’s weight estimation to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value it can help making the update step more conservative.
- *subsample* = [0,1] - subsample ratio of randomly selected training instances used to grow trees,
- *colsample_bytree* = [0,1] - subsample ratio of columns when construction each tree,
- *lambda* = [1] - L2 regularization term on weights
- *alpha* = [0] - L1 regularization term on weights
 
#### 3. Hiperparámetros de aprendizaje

- *objective* = 
  - reg:linear # linear regression,
  - reg:logistic # logistic regression,
  - binary:logistic # logistic regression for binary classification, outputs probability,
  - binary:logitraw #logistic regression for binary classification, outputs score before logistic transformation,
  - count:poisson # poisson regression for count data, outputs mean of poisson distribution,
  - multi:softmax # do multiclass classification using softmax objective,
  - multi:softprob # same as above but outputs probability for each class,
  - rank:pairwise # execute ranking task by minimizing the pairwise loss

- *base_score* = [0.5] - global bias. Initial prediction score for all instances
- *eval_metric* = [rmse|logloss|error|merror|mlogloss|auc|...] - evaluation metric. Default value will be assigned based on the objective. There is possibility of having custom metric.
- *seed* = [0] - seed used for reproducibility


## Bibliografía
* [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), capítulo 8.
* Hands-On Machine Learning with Scikit-Learn and TensorFlow, Aurélien Géron. Capítulos 6 y 7.
* Leo Breiman - Creador de Random Forest. [Excelente web y tutorial]( https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm). 

* Extreme Gradient Boosting (XGBoost). Ganador de muchas competiciones de Kaggle. [Un excelente tutorial sobre Boosting](https://xgboost.readthedocs.io/en/latest/tutorials/model.html).

* Extremely Random Trees.[Random forest donde solo se permite un subconjunto aleatorio para los umbrales. Se elige el mejor umbral de ese conjunto aleatorio.](http://scikit-learn.org/stable/modules/ensemble.html#extremely-randomized-trees).

----
